# General API quickstart

In [ ]:
import warnings

import arviz as az
import matplotlib.pyplot as plt
import numpy as np
import pymc3 as pm
import theano.tensor as tt

warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
%config InlineBackend.figure_format='retina'
az.style.use('arviz-darkgrid')
print(f'Running on PyMC3 v{pm.__version__}')
print(f'Running on ArviZ v{az.__version__}')

## Model creation

Models in PyMC3 are centered around the `Model` class. It has references to all random variables (RVs) and computes the model logp and its gradients. Usually, you would instantiate it as part of a `with` context:

In [ ]:
with pm.Model() as model:
    # Model definition
    pass

We discuss RVs further below but let's create a simple model to explore the `Model` class.

In [ ]:
with pm.Model() as model:
    mu = pm.Normal('mu', mu=0, sigma=1)
    obs = pm.Normal('obs', mu=mu, sigma=1, observed=np.random.randn(100))

In [ ]:
model.basic_RVs

In [ ]:
model.free_RVs

In [ ]:
model.observed_RVs

In [ ]:
model.logp({'mu': 0})

It's worth highlighting the design choice we made with logp. As you can see above, `logp` is being called with arguments, so it's a method of the model instance.  More precisely, it puts together a function based on the current state of the model - or on the state given as an argument to `logp` (see example below).

For diverse reasons, we assume a `Model` instance isn't static. If you need to use `logp` in an inner loop and it needs to be static, simply use something like `logp = model.logp`. Here is an example below - note the caching effect and the speedup.

In [ ]:
%timeit model.logp({mu: 0.1})
logp = model.logp
%timeit logp({mu: 0.1})

## Probability distributions

Every probabilistic program consists of observed and unobserved Random Variables (RVs). Observed RVs are defined via likelihood distributions, while unobserved RVs are defined via prior distributions. In PyMC3, probability distributions are available from the main module space.

In [ ]:
help(pm.Normal)

In [ ]:
dir(pm.distributions.mixture)

### Unobserved Random Variables

Every unobserved RV has the following calling signature: name(str), parameter keyword arguments. Thus, a normal prior can be defined in a model context like this:

In [ ]:
with pm.Model():
    x = pm.Normal('x', mu=0, sigma=1)

As with the model, we can evaluate its logp:

In [ ]:
x.logp({'x': 0})

### Observed Random Variables

Observed RVs are defined just like unobserved RVs but require data to be passed into the observed keyword argument:

In [ ]:
with pm.Model():
    obs = pm.Normal('obs', mu=0, sigma=1, observed=np.random.randn(100))

In [ ]:
obs.logp({'mu': 0})

The `observed` keyword supports values of type `list`, `numpy.ndarray`, `theano`, and `pandas` data structures.

### Deterministic transforms

PyMC3 allows you to freely do algebra in all kinds of ways:

In [ ]:
with pm.Model():
    x = pm.Normal('x', mu=0, sigma=1)
    y = pm.Gamma('y', alpha=1, beta=1)
    plus_2 = x + 2
    summed = x + y
    squared = x ** 2
    sined = pm.math.sin(x)

While these transforms work seamlessly, their results are **not** stored automatically. Thus, if you want to keep track of a transformed variable, you must use `pm.Deterministic`.

In [ ]:
with pm.Model():
    x = pm.Normal('x', mu=0, sigma=1)
    plus_2 = pm.Deterministic('plus_2', x + 2)

Note that `plus_2` can be used in identical to the above, we only tell PyMC3 to keep track of this RV for us.

### Automatic transform of bounded RVs

In order to sample models more efficiently, PyMC3 automatically transforms bounded RVs to be **unbounded**.

In [ ]:
with pm.Model() as model:
    x = pm.Uniform('x', lower=0, upper=1)

When we look at the RVs of the model, we would expect to find `x` there; however:

In [ ]:
model.free_RVs

The variable, `x_interval__`, represents `x` transformed to accept parameter values between `-inf` and `+inf`. In the case of an upper and lower bound, a `LogOdds` transform is applied. Sampling in this transformed space makes it easier for the sampler. PyMC3 also keeps track of the non-transformed bounded parameters. These are common deterministics (see above):

In [ ]:
model.deterministics

When displaying results, PyMC3 will usually hide transformed parameters. You can pass the `include_transformed=True` parameter to many functions to see the transformed parameters that are used for sampling.

You can also turn transforms off:

In [ ]:
with pm.Model() as model:
    x = pm.Uniform('x', lower=0, upper=1, transform=None)

print(model.free_RVs)

Or specify different transformations other than the default:

In [ ]:
import pymc3.distributions.transforms as tr

with pm.Model() as model:
    # Use the default log transform
    x1 = pm.Gamma('x1', alpha=1, beta=1)
    # Specify a different transformation
    x2 = pm.Gamma('x2', alpha=1, beta=1, transform=tr.log_exp_m1)

print(f'The default transformation of x1 is: {x1.transformation.name}')
print(f'The user-specified transformation of x2 is: {x2.transformation.name}'

### Transformed distributions and changes of variables

PyMC3 does **not** provide explicit functionality to transform one distribution to another. Instead, a dedicated distribution is usually created in consideration of optimising performance. However, users can still create transformed distribution by passing the inverse transformation to `transform` `kwarg`. Take the classic textbook example of `LogNormal`: $\mathcal log(y) \sim \mathrm Normal(\mu, \sigma)$.

In [ ]:
class Exp(tr.ElemwiseTransform):
    name = 'exp'

    def backward(self, x):
        return tt.log(x)

    def forward(self, x):
        return tt.exp(x)

    def jacobian_det(self, x):
        return -tt.log(x)

with pm.Model() as model:
    x1 = pm.Normal('x1', 0.0, 1.0, transform=Exp())
    x2 = pm.Lognormal('x2', 0.0, 1.0)

lognormal = model.named_vars['x1_exp__']
lognorm2 = model.named_vars['x2']

_, ax = plt.subplots(figsize=(5, 3))
x = np.linspace(0.0, 10.0, 100)
ax.plot(
    x,
    np.exp(lognormal.distribution.logp(x).eval()),
    '--',
    alpha=0.5,
    label='log(y) ~ Normal(0, 1)',
)
ax.plot(
    x,
    np.exp(lognorm2.distribution.logp(x).eval()),
    alpha=0.5,
    label='y ~ lognormal(0, 1)',
)
plt.legend()

Notice from above that the named variable `x1_exp__` in the `model` is Lognormal distributed.

Using a similar approach, we can create ordered RVs following some distribution. For example, we can combine the ordered transformation and logodds transformation using `Chain` to create a 2D RV that satisfy $\mathcal x1, x2 \sim \mathrm Uniform(0, 1) \mathcal\ and\ x1 < x2$.

In [ ]:
Order = tr.Ordered()
Logodd = tr.LogOdds()
chain_tran = tr.Chain([Logodd, Order])

with pm.Model() as m0:
    x = pm.Uniform('x', 0.0, 1.0, shape=2, transform=chain_tran, testval=[0.1, 0.9])
    trace = pm.sample(5000, tune=1000, progressbar=False, return_inferencedata=False)

In [ ]:
_, ax = plt.subplots(1, 2, figsize=(10, 5))
for ivar, varname in enumerate(trace.varnames):
    ax[ivar].scatter(trace[varname][:, 0], trace[varname][:, 1], alpha=0.01)
    ax[ivar].set_xlabel(f'{varname}[0]')
    ax[ivar].set_ylabel(f'{varname}[1]')
    ax[ivar].set_title(varname)
plt.tight_layout()

### List of RVs / higher-dimensional RVs

Above we have seen how to create scalar RVs. In many models, you want multiple RVs. There is a tendency (mainly inherited from PyMC 2.x) to create a list of RVs, like:

In [ ]:
with pm.Model():
    # bad
    x = [pm.Normal(f'x_[{i}]', mu=0, sigma=1) for i in range(10)]

However, even though this works, it is quite slow and not recommended. Instead, use the `shape` kwarg.

In [ ]:
with pm.Model() as model:
    # good
    x = pm.Normal('x', mu=0, sigma=1, shape=10)

`x` is now a random vector of length 10. We can index into it or do linear algebra operations on it.

In [ ]:
with model:
    y = x[0] * x[1]  # full indexing is supported
    x.dot(x.T)  # linear algebra is supported

### Initialization with test values

While PyMC3 tries to automatically initialize models, it is sometimes helpful to define initial values for RVs. This can be done via the `testval` kwarg:

In [ ]:
with pm.Model():
    x = pm.Normal('x', mu=0, sigma=1, shape=5)

x.tag.test_value

In [ ]:
with pm.Model():
    x = pm.Normal('x', mu=0, sigma=1, shape=5, testval=np.random.randn(5))

x.tag.test_value

This technique is quite useful to identify problems with model specification or initialization.